In [173]:
import gc
from copy import deepcopy
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, LearningRateScheduler, TensorBoard
from keras import backend as K

from libs.util import random_mask
from libs.pconv_model_first_resid import PConvUnet
# from libs.pconv_model_UNet import PConvUnet
from libs.properties import properties

# Settings
MAX_BATCH_SIZE = 32

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [174]:
properties_dict = properties()
length = properties_dict["length"]

In [175]:
matrix_df = pd.read_csv('./data/trafficV_M.csv', index_col=0, parse_dates=True)

In [176]:
# matrix_df = np.array(matrix_df)

# print(matrix_df.shape)
# print("%d bytes" % (matrix_df.size * matrix_df.itemsize))
# print(np.isnan(matrix_df).sum())

In [177]:
# # 单独测试week_ago, mintue_ago
# week_delta = pd.Timedelta(1, unit='W')
# minute_delta = pd.Timedelta(15, unit='m')


# channel_num = 3
# smooth_time = channel_num-1
# set_up_time = week_delta

# train_df = matrix_df.truncate(before=matrix_df.index.min() + set_up_time)
# train_week_ago_df = matrix_df.loc[train_df.index - week_delta]
# train_minute_ago_df = matrix_df.loc[train_df.index - minute_delta]

# train_df = np.array(train_df).reshape(-1, length, length, 1)
# train_week_ago_df = np.array(train_week_ago_df).reshape(-1, length, length, 1)
# train_minute_ago_df = np.array(train_minute_ago_df).reshape(-1, length, length, 1)


# train_array = np.concatenate((train_df, train_minute_ago_df, train_week_ago_df), axis=3)
# X_train, X_val = train_test_split(train_array, test_size = 0.1, random_state=42)

In [178]:
def createTrainArray(week_history_num=0, minute_history_num=0):
    week_delta_list = [pd.Timedelta(i+1, unit='W') for i in range(week_history_num)]
    minute_delta_list = [pd.Timedelta((i+1)*15, unit='m') for i in range(minute_history_num)]
    delta_list = week_delta_list+minute_delta_list
    print(delta_list)
    
    set_up_time = pd.Timedelta(week_history_num, unit='W')
    train_df = matrix_df.truncate(before=matrix_df.index.min() + set_up_time)
    
    train_ago_array_tuple = tuple([np.array(matrix_df.loc[train_df.index - i]).reshape(-1, length, length, 1) for i in delta_list])
    train_df = np.array(train_df).reshape(-1, length, length, 1)
    train_array = np.concatenate((train_df,)+train_ago_array_tuple, axis=3)
    print(train_array.shape)
    return train_array

In [179]:
week_history_num = 1
minute_history_num = 1

channel_num = week_history_num +minute_history_num +1
smooth_time = channel_num-1

train_array = createTrainArray(week_history_num, minute_history_num)
X_train, X_val = train_test_split(train_array, test_size = 0.1, random_state=42)

[Timedelta('7 days 00:00:00'), Timedelta('0 days 00:15:00')]
(16704, 32, 32, 3)


In [180]:
epoch_steps = X_train.shape[0] // MAX_BATCH_SIZE
val_steps = X_val.shape[0] // MAX_BATCH_SIZE
epoch_steps, val_steps

(469, 52)

In [181]:
class DataGenerator(ImageDataGenerator):
    def flow(self, X, *args, **kwargs):
        while True:
            
            # Get augmentend image samples
            ori = next(super().flow(X, *args, **kwargs))
    
            # Get masks for each image sample
            mask = np.stack([random_mask(ori.shape[1], ori.shape[2], size=0.1, channels=channel_num, smooth_time=smooth_time) for _ in range(ori.shape[0])], axis=0)

            # Apply masks to all image sample
            masked = deepcopy(ori)
#             print(masked.shape)
            masked_mean = masked[mask==1].mean()
            masked[mask==0] = masked_mean

            # Yield ([ori, masl],  ori) training batches
#             print(masked.shape, ori.shape)
            gc.collect()
            yield [masked, mask], ori[:,:,:,:1]
            
train_datagen = DataGenerator()
train_generator = train_datagen.flow(
    X_train, batch_size=MAX_BATCH_SIZE
)

# Create validation generator
val_datagen = DataGenerator()
val_generator = val_datagen.flow(
    X_val, batch_size=MAX_BATCH_SIZE
)


In [182]:
# np.array(next(train_generator)[1])[:,:,:,:1].shape
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=1, mode='auto')

In [183]:
model = PConvUnet(img_rows=length, img_cols=length, channels=channel_num)
optimizer = model.get_optimizer()
def scheduler(epoch):
    if epoch>4 and epoch % 1 == 0 and epoch != 0:
        lr = K.get_value(optimizer.lr)
        if lr>0.00005:
            if epoch%4==0:
                K.set_value(optimizer.lr, lr * 0.6)
            else:
                K.set_value(optimizer.lr, lr * 0.8)
        print(K.get_value(optimizer.lr))
    return K.get_value(optimizer.lr)

reduce_lr = LearningRateScheduler(scheduler)

(?, 32, 32, 1)


In [184]:
model.fit(
    train_generator, 
    validation_data=val_generator,
    steps_per_epoch = epoch_steps,
    validation_steps = val_steps,
    epochs = 10,
    callbacks=[reduce_lr]
)

Epoch 1/1
469/469 [==============================] - 218s 465ms/step - loss: 730931.3238 - val_loss: 232926.7500
Epoch 2/2
469/469 [==============================] - 216s 461ms/step - loss: 198903.4773 - val_loss: 168960.3134
Epoch 3/3
469/469 [==============================] - 213s 454ms/step - loss: 149919.7399 - val_loss: 148197.3324
Epoch 4/4
469/469 [==============================] - 210s 447ms/step - loss: 137289.1406 - val_loss: 128320.8822
Epoch 5/5
469/469 [==============================] - 210s 449ms/step - loss: 125006.4074 - val_loss: 6262788.7788
Epoch 6/6
0.0032000002
469/469 [==============================] - 215s 458ms/step - loss: 111767.2877 - val_loss: 135548.7108
Epoch 7/7
0.0025600002
469/469 [==============================] - 214s 456ms/step - loss: 101959.5603 - val_loss: 169000.9497
Epoch 8/8
0.0020480002
469/469 [==============================] - 212s 452ms/step - loss: 97231.5616 - val_loss: 113157.8514
Epoch 9/9
0.0012288002
469/469 [=========================

In [194]:
mask_shape = X_val.shape
mask = np.stack([random_mask(mask_shape[1], mask_shape[2], size=0.1, channels=channel_num, smooth_time=smooth_time) for _ in range(mask_shape[0])], axis=0)

masked = deepcopy(X_val)
masked_mean = masked[mask==1].mean()
masked[mask==0] = masked_mean

y_pred = model.predict([masked, mask])
y_true = X_val[:,:,:,:1]

y_true = (1-mask[:,:,:,:1])*y_true
y_pred = (1-mask[:,:,:,:1])*y_pred

In [186]:
def l2(y_true, y_pred):
    return np.sum(np.mean(np.square(y_true - y_pred), axis=0))

def l1(y_true, y_pred):
    return np.sum(np.mean(np.abs(y_true - y_pred), axis=0))

In [195]:
l2(y_true, y_pred)

51992.075384870026

In [196]:
np.min(np.sum(np.sum(np.sum(np.abs(y_true - y_pred), axis=3), axis=2), axis=1))

293.287428779602

In [189]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_58 (Conv2D)              (None, 32, 32, 32)   896         input_7[0][0]                    
__________________________________________________________________________________________________
activation_64 (Activation)      (None, 32, 32, 32)   0           conv2d_58[0][0]                  
__________________________________________________________________________________________________
average_pooling2d_10 (AveragePo (None, 16, 16, 32)   0           activation_64[0][0]              
__________________________________________________________________________________________________
batch_norm

In [190]:
list = []
for _ in range(20):
    list.append(model.evaluate_generator(val_generator, 1))
list
# model.evaluate_generator(val_generator, 100)

[104467.2421875,
 71870.6015625,
 84542.78125,
 65168.51953125,
 100770.1953125,
 57086.05859375,
 82728.9609375,
 91935.3515625,
 67244.125,
 67487.578125,
 101025.453125,
 90444.5390625,
 85946.4453125,
 53714.6171875,
 83675.03125,
 68728.0703125,
 50792.859375,
 116649.2890625,
 83531.1484375,
 78308.1015625]

In [191]:
# import random
# test_num = random.randint(0, 200)
# test = deepcopy(X_train[test_num,np.newaxis,:])

# test_mask = random_mask(test.shape[1], test.shape[2], size=0.1, channels=channel_num, smooth_time=smooth_time)
# test_mask = test_mask[np.newaxis,:]

# test_mask[0,:,:,0].shape
# test[test_mask==0] = test.mean()

# # test_mask.shape
# # plt.imshow(test[0,:,:,0]*255)
# test_res = model.predict([test, test_mask])
# # np.sum((test-test_res)**2)
# np.sum((test[0,:,:,0][test_mask[0,:,:,0] == 0] - test_res[0,:,:,0][test_mask[0,:,:,0] == 0])**2)